# Machine Learning with Spark ML

### In this notebook, we will explore machine learning using Spark ML. We will exploit Spark ML's high-level APIs built on top of DataFrames to create and tune machine learning pipelines. Spark ML Pipelines enable combining multiple algorithms into a single pipeline or workflow. We will utilize Spark ML's feature transformers to convert, modify and scale the features that will be used to develop the machine learning model. Finally, we will evaluate and cross validate our model to demonstrate the process of determining a best fit model and load the results in the database.

### We are using machine learning to try to predict records that a human has not seen or vetted before. We will use these predictions to sort the highest priority records for a human to look at. We will use as a training set for the algorithm fake data that has been vetted by an analyst as high, medium or low.¶

### We will use generated travel data that has been examined for patterns of Human Trafficking from a DB2 table to do the machine learning.  

## Table of contents

1. [Setup](#copy)
1. [Import Libraries](#install)
1. [Read from Object Storage](#object)
1. [Transform the data](#transform)
1. [Feature Engineering](#engineering)
1. [Model the data](#model)
1. [Setup the Pipeline](#pipeline)
1. [Train the model](#train)
1. [Evaluate results](#evaluate)
1. [Hyperparameter Tuning](#tuning)
1. [Score the records](#score)
1. [Insert Credentials](#credentials)
1. [Write Results](#write)
1. [Create New Version](#version2)
1. [Schedule Job](#schedule)
1. [Revert to Version](#revert)
1. [Even More Help](#help)


## Setup

In order to interface to the WatsonMachineLearning service we need to obtain an api key and find the location of the service. The procedure is written in the lab instructions. 

In [ ]:
api_key = "PASTE API KEY HERE"
location = "PASTE LOCATION HERE"
projectid = "PASTE PROJECT ID HERE"



## Install pixiedust library

In [ ]:
# ignore the errors in the install
!pip install pixiedust

## Verify Spark version and existence of Spark

In [ ]:
print('The spark version is {}.'.format(spark.version))

<a id="install"></a>
##  Import the required libraries

In [ ]:
#Imports for Spark
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.feature import Bucketizer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import NaiveBayes, DecisionTreeClassifier,RandomForestClassifier
from pyspark.sql.functions import year
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Imports for pixiedust
from pixiedust.display import *

<a id="object"></a>
## Read Data Asset - female_human_trafficking_shaped_100 - See Lab Instructions

In [ ]:
# Insert female_human_trafficking_shaped_100 SparkDataFrame in this cell after the comments
# make CERTAIN that data_df_n where n is a number is changed to trafficking_df
#Put cursor on the next line to Insert to code # @hidden_cell# @hidden_cell



<a id="transform"></a>
## Identify our labels and transform 

We will use the 'VETTING_LEVEL' column as a label for training the machine learning model.  This is where our analyst has marked the data as vetted.  

Spark ML requires that that the labels are data type Double, so we will cast the  column as Double.

withColumn() is a Spark SQL way to manipulate a dataframe.  Since an RDD is immutable, we create a new RDD each time we transform.  This code creates a new column VettingTemp and sets it to the values in "VETTING_LEVEL" cast to a Double.    It then drops column VETTING_LEVEL and renames column VettingTemp to VETTING_LEVEL.

We will also do a similar manipulation for the COUNTRIES_VISITED_COUNT

In [ ]:
DataWithLabels = (trafficking_df.withColumn("VettingTemp", trafficking_df["VETTING_LEVEL"]
    .cast("Double")).drop("VETTING_LEVEL").withColumnRenamed("VettingTemp", "VETTING_LEVEL"))

DataWithLabels = (trafficking_df.withColumn("CountriesVisitedCountTemp", trafficking_df["COUNTRIES_VISITED_COUNT"]
    .cast("Double")).drop("COUNTRIES_VISITED_COUNT").withColumnRenamed("CountriesVisitedCountTemp", "COUNTRIES_VISITED_COUNT"))

We want to use year of birth intead of date of birth in our learning.  

Another way to transform an rdd in Spark is using SQL Syntax.  Here, we will be adding a new field, BIRTH_YEAR to our vetting set.  We will also just select the fields we need.

In [ ]:
DataWithLabels.createOrReplaceTempView("VettingData")
AllVettingData = sqlContext.sql ("SELECT VETTING_LEVEL, NAME, Category, COUNTRIES_VISITED_COUNT, PASSPORT_COUNTRY, year(BIRTH_DATE) as BIRTH_YEAR, 1 as Counter FROM VettingData")
FilteredVettingData = AllVettingData.filter("VETTING_LEVEL==100")
FilteredVettingData.count()

Use pixiedust to visually explore the data.

In [ ]:
display(AllVettingData)

Now, let's look at the data we have:

VETTING_LEVEL is in four different statuses:

    10 - HIGH
    
    20 - MEDIUM
    
    30 - LOW
    
    100 - Unlabeled


Print the total number of vetting statuses 

In [ ]:
print('The number of rows labeled high is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 20).count()))
print('The number of rows labeled low is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 30).count()))
print('The number of unlabeled rows is {}.'.format(AllVettingData.filter(AllVettingData['VETTING_LEVEL'] == 100).count()))

The majority of the data has not been labeled (VETTING_LABEL=100 means unvetted).  We can not use it for our training data, so filter it out.
Print the total number of rows.

In [ ]:
LabeledVettingData=AllVettingData.filter("VETTING_LEVEL != 100")
LabeledVettingData.count()

In [ ]:
display(LabeledVettingData)

<a id="engineering"></a>
## Feature Engineering.
### A feature is the elements of the data that we are using in our learning.  We need to transform each one of our features into a format that SparkML can use it.
More about the choices for feature engineering can be found here:
http://spark.apache.org/docs/2.0.0/ml-features.html#stringindexer


The first thing we will do is transform our labels (VETTING_LEVEL) into a format that we can use in the algorithm, and then get back to 'human readable' from in the end. The ML models require that the labels are in a column called 'label'.    The converter helps us transform these back in the end.



In [ ]:
labelIndexer = StringIndexer(inputCol="VETTING_LEVEL", outputCol="label", handleInvalid="error")
labelModel = labelIndexer.fit(LabeledVettingData)
converter = IndexToString(inputCol="prediction", outputCol="predCategory", labels=labelModel.labels)

Next, we will process all of the features we will use. While there are a variety of choices for transforming elements, we will treat each as a String using the StringIndexer.

StringIndexer is a transformer that encodes a string column to a column of indices. The indices are ordered by value frequencies, so the most frequent value gets index 0. If the input column is numeric, it is cast to string first.

For our vetting dataset, we are interested in all string based features so we will use the StringIndexer for them.  We need to use 'handleInvalid="skip"' because not all values have been validated in our vetting set.  That means the algorithms will skip these records.

In [ ]:
categoryIndexer = StringIndexer(inputCol="Category", outputCol="categoryIndex", handleInvalid="skip")
countryIndexer = StringIndexer(inputCol="PASSPORT_COUNTRY", outputCol="countryIndex", handleInvalid="skip")
yearOfBirthIndexer = StringIndexer(inputCol="BIRTH_YEAR", outputCol="birthYearIndex", handleInvalid="skip")

Now, put all of our features into a simple array using a VectorAssembler.

Note that COUNTRIIES_VISITED_COUNT is already a numeric, so we can just put that in the array as is.


In [ ]:
vecAssembler = VectorAssembler(inputCols=["categoryIndex","countryIndex","birthYearIndex", "COUNTRIES_VISITED_COUNT"], outputCol="features")

Normalizer will help us normalize the features into a standard frmat.  It can help us improve the behavior of the learning algorithms.


In [ ]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)

<a id="model"></a>
## Declare the model that we want to use

The model here is Naive Bayes.  It will output each prediction into a 'prediction' column.  Naive Bayes  is a probabistic model that learns based on previous decisions.  We will take a best guess at the paramater 'smoothing'- SparkML will help us tune it later!


In [ ]:
nb = NaiveBayes(smoothing=1.0, featuresCol="normFeatures",modelType="multinomial", labelCol="label", predictionCol="prediction")

<a id="pipeline"></a>
## Setup the Pipeline

The pipeline is the guts of the algorithm that strings all the work we've done together.

The stages are run in order and the input DataFrame is transformed as it passes through each stage.   First, comes the feature transformations, then the assembler to put them togather into one DF.  We pass that into the model. 

In machine learning, it is common to run a sequence of algorithms to process and learn from data, so this can get as complex as we want to make it!

In [ ]:
pipeline = Pipeline(stages=[labelIndexer,categoryIndexer,countryIndexer, yearOfBirthIndexer, vecAssembler, normalizer, nb, converter])

<a id="train"></a>
## Train the model

We will split it into training data which is marked and test data which will be used to test the efficiency of the algorithms.

It is common to split the split up the data randomly into 70% for training and 30% for testing.  If we were to use a bigger training set, we might use an 80% / 20% split.

In [ ]:
train, test = LabeledVettingData.randomSplit([70.0,30.0], seed=152345)
train.cache()
test.cache()
print('The number of records in the training data set is {}.'.format(train.count()))
print('The number of rows labeled high is {}.'.format(train.filter(train['VETTING_LEVEL'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(train.filter(train['VETTING_LEVEL'] == 20).count()))
print('The number of rows labeled low is {}.'.format(train.filter(train['VETTING_LEVEL'] == 30).count()))
print('')

print('The number of records in the test data set is {}.'.format(test.count()))
print('The number of rows labeled high is {}.'.format(test.filter(test['VETTING_LEVEL'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(test.filter(test['VETTING_LEVEL'] == 20).count()))
print('The number of rows labeled low is {}.'.format(test.filter(test['VETTING_LEVEL'] == 30).count()))

Fit the pipeline to the training data.  This will run the data through the algorithm to train it based on our labled data.

In [ ]:
# Fit the pipeline to the training data assigning the result to a variable called 'model'.
model = pipeline.fit(train)

Make predictions on records in the Test data set.  This will test the model based on the 30% data we have left in reserve.  Keep in mind that the model has not seen the data in the test data set.

In [ ]:
# Make predictions on the test data assigning the result to a variable called 'predictions'.
predictions = model.transform(test)

<a id="evaluate"></a>
## Show and Evaluate Results

Note that we only got a small sample of the results back because we have a very small amount of training data. 

In [ ]:
predictions.count()

SparkML has automated ways to look at result quality called Evaluators.  More information can be found here:
http://spark.apache.org/docs/latest/mllib-evaluation-metrics.html

For simplicity here, we will use a a common evaluation method called Reciever Operator Characteristic.  This genenerally is used for binary classifiers, but we will use it because we only have 3 levels of prediction.

The curve is created by plotting the true positive rate against the false positive rate at various threshold settings. The ROC curve is thus the sensitivity as a function of fall-out. The area under the ROC curve is useful for comparing and selecting the best machine learning model for a given data set. A model with an area under the ROC curve score near 1 has very good performance. A model with a score near 0.5 is about as good as flipping a coin.

In [ ]:
evaluator = BinaryClassificationEvaluator().setLabelCol("label").setMetricName("areaUnderROC")
print('Area under the ROC curve = {}.'.format(evaluator.evaluate(predictions)))

<a id="tuning"></a>
## Automatic Algorithm Tuning - Also Called  Hyperparameter Tuning


Spark ML algorithms provide many hyperparameters for tuning models. These hyperparameters are distinct from the model parameters being optimized by Spark ML itself.  Hyperparameter tuning is accomplished by choosing the best set of parameters based on model performance on test data that the model was not trained with. All combinations of hyperparameters specified will be tried in order to find the one that leads to the model with the best evaluation result.

First we will build a paramater grid to tell SparkML what to change in its testing.  Note that we are changing all the paramaters we setup in our pipeline before - the 'smoothing' in our model, and the normalizer parameter.

In [ ]:
paramGrid = (ParamGridBuilder().addGrid(nb.smoothing, [0.50,0.75,1.0])
                 .addGrid(normalizer.p, [1.0, 1.25, 1.5]).build())

Now, create a cross validator to tune the pipeline with the generated parameter grid.  Cross-validation attempts to fit the underlying estimator with user-specified combinations of parameters, cross-evaluate the fitted models, and output the best one.  

In [ ]:
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(5)

Next, we will run the models through the grid we set above.  It runs a cross evaluation of the ML Pipeline to find the best model.  Note that since runs the model several times, it takes a few minutes to run.

In [ ]:
cvModel = cv.fit(train)
print('Area under the ROC curve for best fitted model = {}.'.format(evaluator.evaluate(cvModel.transform(test))))

Let's see what improvement we achieve by tuning the hyperparameters using cross-evaluation 

In [ ]:
print('Area under the ROC curve for non-tuned model = {}.'.format(evaluator.evaluate(predictions)))
print('Area under the ROC curve for best fitted model = {}.'.format(evaluator.evaluate(cvModel.transform(test))))
print('Improvement = {0:0.2f}%'.format((evaluator.evaluate(cvModel.transform(test)) - evaluator.evaluate(predictions)) *100 / evaluator.evaluate(predictions)))

<a id="score"></a>
## Score the remaining records that were unscored

First, we want to only get the unvetted records.

In [ ]:
NewVettingData=AllVettingData.filter("VETTING_LEVEL == 100")

Next, transform the new model with the new vetting records

In [ ]:
newPreds = cvModel.transform(NewVettingData)

 Show the data we have predicted and some of the fields in the data.  

In [ ]:
newPreds.select("prediction", "predCategory", "probability", "NAME", "COUNTRIES_VISITED_COUNT", "PASSPORT_COUNTRY" ).show()

Remember that VETTING_LEVEL is in three different statuses:


10- HIGH

20- MEDIUM

30 - LOW


Let's print the total number of vetting statuses that we predicted.  The actual predicted data is low because we only have a few vetted records.  Remember that we had to 'skip' and features that were not in our trained data, so if we didn't have someone who was born in a certain year in our training data, we won't be able to predict a result.

In [ ]:
print('The number of records in the unvetted data set is {}.'.format(newPreds.count()))
print('The number of rows labeled high is {}.'.format(newPreds.filter(newPreds['predCategory'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(newPreds.filter(newPreds['predCategory'] == 20).count()))
print('The number of rows labeled low is {}.'.format(newPreds.filter(newPreds['predCategory'] == 30).count()))

## Try Random Forest Model 

In [ ]:
rf = RandomForestClassifier().setLabelCol("label").setFeaturesCol("normFeatures").setNumTrees(15)
# Create new Pipeline using the RandomForest model and all the same feature transformers used above for logistic regression
pipelineRF = Pipeline(stages=[labelIndexer,categoryIndexer,countryIndexer, yearOfBirthIndexer, vecAssembler, normalizer, rf, converter])

# Train model.
modelRF = pipelineRF.fit(train)

# Make predictions.
predictionsRF = modelRF.transform(test)

evaluator = BinaryClassificationEvaluator().setLabelCol("label").setMetricName("areaUnderROC")
print('Area under the ROC curve = {}.'.format(evaluator.evaluate(predictionsRF)))

newRFPreds = modelRF.transform(NewVettingData)

print('The number of records in the unvetted data set is {}.'.format(newRFPreds.count()))
print('The number of rows labeled high is {}.'.format(newRFPreds.filter(newRFPreds['predCategory'] == 10).count()))
print('The number of rows labeled medium is {}.'.format(newRFPreds.filter(newRFPreds['predCategory'] == 20).count()))
print('The number of rows labeled low is {}.'.format(newRFPreds.filter(newRFPreds['predCategory'] == 30).count()))


## Save Model - This section below demonstrates the Watson Machine Language API which allows you to programmatically save the model that was trained to a model repository.  The model can then be deployed for use in a production application.  





In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)
client.set.default_project(projectid)

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("spark-mllib_3.0-py37")
software_spec_uid

In [ ]:
metadata = {
        client.repository.ModelMetaNames.NAME: 'FHT_Spark',
        client.repository.ModelMetaNames.DESCRIPTION :'',
        client.repository.ModelMetaNames.TYPE: "mllib_2.4",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : "36507ebe-8770-55ba-ab2a-eafe787600e9"
}
        

## After the cell below is executed, a model entry "FHT Spark" will appear in the Project Model section. 

In [ ]:
client.repository.store_model(modelRF, meta_props=metadata, training_data=train, pipeline=pipelineRF)

## Please return to the lab instructions